# import and load data set

In [1]:
import geopandas as gpd
import pandas as pd
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
import numpy as np


from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

## Read csvs

train_df = gpd.read_file('train.geojson', index_col=0)
test_df = gpd.read_file('test.geojson', index_col=0)

C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(
C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(


In [31]:
## Filtering column "mail_type"
## Adding total construction time

import pandas as pd
import numpy as np
PRE_CONSTRUCTION_STATUSES = {
    "Greenland",
    "Land Cleared",
    "Prior Construction",
    "Excavation",
    "Materials Dumped",
    "Materials Introduced",
    "Construction Started",
}

POST_CONSTRUCTION_STATUSES = {
    "Construction Midway",
    "Construction Done",
    "Operational",
}

import numpy as np

def add_geometry_features(gdf, metric_epsg=6933):
    """
    gdf: GeoDataFrame en EPSG:4326
    Ajoute:
      - polygon_area_m2
      - polygon_perimeter_m
      - compactness (sans unité)
    """
    # 1) reprojection en CRS métrique (mètres)
    gdf_m = gdf.to_crs(epsg=metric_epsg)

    # 2) calculs géométriques en mètres
    gdf = gdf.copy()
    gdf["polygon_area_m2"] = gdf_m.geometry.area
    gdf["polygon_perimeter_m"] = gdf_m.geometry.length

    # 3) compacité = 4πA / P²
    gdf["compactness"] = (
        4 * np.pi * gdf["polygon_area_m2"] / (gdf["polygon_perimeter_m"] ** 2)
    )
    gdf.loc[gdf["polygon_perimeter_m"] == 0, "compactness"] = np.nan

    return gdf

def add_max_gap_between_sets(
    df,
    date_cols,     # ex: ["date0", ..., "date4"]
    status_cols    # ex: ["change_status_date0", ..., "change_status_date4"]
):
    out = df.copy()

    dates = out[date_cols].apply(pd.to_datetime, errors="coerce", utc=True)
    statuses = out[status_cols]

    def row_max_gap(row_dates, row_statuses):
        set1_dates = []
        set2_dates = []

        for d, s in zip(row_dates, row_statuses):
            if pd.isna(d) or pd.isna(s):
                continue
            if s in PRE_CONSTRUCTION_STATUSES:
                set1_dates.append(d)
            elif s in POST_CONSTRUCTION_STATUSES:
                set2_dates.append(d)

        if not set1_dates or not set2_dates:
            return 0.0

        # max |t2 - t1|
        max_gap = max(
            abs((t2 - t1).total_seconds())
            for t1 in set1_dates
            for t2 in set2_dates
        )

        return max_gap / 86400.0  # en jours

    out["pre_post_construction_time_gap_days"] = [
        row_max_gap(d, s)
        for d, s in zip(dates.to_numpy(), statuses.to_numpy())
    ]

    return out

date_cols = [f"date{i}" for i in range(5)]
status_cols = [f"change_status_date{i}" for i in range(5)]

train_df = add_geometry_features(train_df)
test_df = add_geometry_features(test_df)


train_df = add_max_gap_between_sets(train_df, date_cols, status_cols)
test_df  = add_max_gap_between_sets(test_df, date_cols, status_cols)

#
# date_cols = [f'date{i}' for i in range(5)]
# status_cols = [f'change_status_date{i}' for i in range(5)]
#
# def get_const_time(df):
#     dt_cols = []
#     for i, c in enumerate(date_cols):
#         dtc = f"dt_{i}"
#         df[dtc] = pd.to_datetime(df[c], errors="coerce", utc=True)  # utc pour être stable
#         dt_cols.append(dtc)
#
#     for i, c in enumerate(status_cols):
#         df[f"change_status_date{i}"].apply(lambda x: status_cols[x])

C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\shapely\measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\shapely\measurement.py:196: RuntimeWarning: invalid value encountered in length
  return lib.length(geometry, **kwargs)
C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\shapely\measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\shapely\measurement.py:196: RuntimeWarning: invalid value encountered in length
  return lib.length(geometry, **kwargs)
C:\Users\Ely Cheikh Abass\AppData\Local\Temp\ipykernel_7620\123306701.py:55: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (th

In [0]:

# train_x_d = np.asarray(train_df['date0'])
feature_cols = [
    "pre_post_construction_time_gap_days",
    "polygon_area_m2",
    "polygon_perimeter_m",
    "compactness",
]

# X (numpy arrays)
train_x = train_df[feature_cols].to_numpy(dtype=float)
test_x  = test_df[feature_cols].to_numpy(dtype=float)

# y (labels encodés)
train_y = train_df["change_type"].map(change_type_map).to_numpy()

print(train_x.shape, train_y.shape, test_x.shape)

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold

# =========================
# PARAMÈTRES
# =========================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# pipeline FIXE : Imputer -> Scaler -> SVC
svc_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # gère NaN
    ("scaler", StandardScaler()),                   # SVC a besoin de scaling
    ("model", SVC(kernel="rbf", C=1))
])

# =========================
# FEATURES AVANT (baseline)
# =========================
baseline_features = [
    "polygon_area_m2"   # <-- mets EXACTEMENT ce que tu utilisais avant
]

X_old = train_df[baseline_features].to_numpy(dtype=float)
y = train_df["change_type"].map(change_type_map).to_numpy()

scores_old = cross_val_score(
    svc_pipe, X_old, y,
    cv=cv, scoring="f1_macro", n_jobs=-1
)

# =========================
# FEATURES APRÈS (nouvelles)
# =========================
new_features = [
    "pre_post_construction_time_gap_days",
    "polygon_area_m2",
    "polygon_perimeter_m",
    "compactness",
]

X_new = train_df[new_features].to_numpy(dtype=float)

scores_new = cross_val_score(
    svc_pipe, X_new, y,
    cv=cv, scoring="f1_macro", n_jobs=-1
)

# =========================
# RÉSULTATS
# =========================
print("===== AVANT =====")
print("F1 macro mean :", scores_old.mean())
print("F1 macro std  :", scores_old.std())

print("\n===== APRÈS =====")
print("F1 macro mean :", scores_new.mean())
print("F1 macro std  :", scores_new.std())

print("\n===== GAIN =====")
print("Gain absolu :", scores_new.mean() - scores_old.mean())
print("Gain relatif (%) :", (scores_new.mean() / scores_old.mean() - 1) * 100)


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ("model", LogisticRegression())
])

param_grid = [
    {
        "model": [LogisticRegression(max_iter=1000)],
        "model__C": [0.1, 1, 10]
    },
    {
        "model": [SVC()],
        "model__C": [0.1, 1],
        "model__kernel": ["rbf"]
    },
    {
        "model": [RandomForestClassifier()],
        "model__n_estimators": [100, 300]
    }
]

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    verbose=True
)

grid.fit(train_x, train_y)
print(grid.best_score_, grid.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
0.1134029891816655 {'model': SVC(), 'model__C': 1, 'model__kernel': 'rbf'}


# Predict and create csv prediction

In [5]:
pred_y = grid.predict(test_x)
print(f"{accuracy_score(train_y, pred_y)}")

ValueError: Found input variables with inconsistent numbers of samples: [296146, 120526]

In [ ]:

## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['change_type'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')